# Day 9: Smoke Basin

In [1]:
from pathlib import Path
from math import inf
from itertools import chain

from aoc2021.util import read_as_list, transpose

## Puzzle input data

In [2]:
parse_input = lambda line: [int(c) for c in line.rstrip()]
# Test data.
tdata = list(map(parse_input,[
    '2199943210',
    '3987894921',
    '9856789892',
    '8767896789',
    '9899965678',
]))

# Input data.
data = read_as_list(Path('./day09-input.txt'), func=parse_input)
print(data[:2])

[[6, 6, 8, 9, 9, 2, 1, 0, 1, 3, 4, 9, 8, 7, 8, 9, 9, 0, 2, 3, 4, 6, 7, 8, 9, 2, 0, 1, 9, 9, 7, 5, 6, 7, 8, 9, 3, 2, 9, 4, 2, 1, 2, 3, 4, 9, 4, 2, 1, 2, 3, 4, 7, 8, 9, 4, 3, 9, 8, 9, 8, 7, 6, 5, 4, 9, 8, 6, 6, 5, 4, 5, 7, 6, 8, 9, 9, 6, 4, 9, 8, 7, 6, 5, 4, 3, 1, 2, 3, 5, 7, 8, 9, 6, 5, 6, 5, 6, 7, 8], [5, 5, 7, 8, 8, 9, 2, 1, 5, 4, 9, 8, 7, 6, 9, 9, 8, 9, 4, 5, 5, 9, 8, 9, 4, 3, 9, 9, 8, 7, 5, 4, 7, 8, 7, 8, 9, 9, 8, 9, 9, 0, 1, 2, 6, 8, 9, 9, 2, 3, 4, 5, 6, 9, 6, 5, 9, 8, 7, 6, 9, 8, 9, 6, 9, 8, 7, 5, 4, 3, 2, 7, 4, 5, 9, 7, 8, 9, 3, 2, 9, 8, 7, 7, 5, 2, 0, 1, 2, 5, 6, 9, 8, 7, 4, 3, 4, 7, 8, 9]]


## Puzzle answers
### Part 1

In [3]:
def local_min_mask(xs: list[int]) -> list[bool]:
    return [v < min(l,r) for v,l,r in zip(xs, [inf]+xs, xs[1:]+[inf])]


def low_points(data: list[list[int]]) -> list[int]:
    rows_mask = map(local_min_mask, data)
    cols_mask = transpose(map(local_min_mask, transpose(data)))
    lows_mask = [all(bs) for rs,cs in zip(rows_mask, cols_mask) for bs in zip(rs,cs)]
    return [x for x,b in zip(chain.from_iterable(data), lows_mask) if b]


def total_risk(lps: list[int]) -> int:
    return sum(lps) + len(lps)


assert local_min_mask([1,2,3,2,5,5]) == [True,False,False,True,False,False]
assert len(low_points(tdata)) == len([1,0,5,5])
assert sum(low_points(tdata)) == sum([1,0,5,5])
assert total_risk(low_points(tdata)) == 15

In [4]:
n = total_risk(low_points(data))
print(f'The sum of the risk levels of all low points on the heightmap: {n}')

The sum of the risk levels of all low points on the heightmap: 500


### Part 2